# 如何添加跨线程持久化（函数式API）

!!!信息“先决条件”

本指南假设您熟悉以下内容：

- [功能 API](../../concepts/function_api/)
- [持久性](../../概念/持久性/)
- [记忆](../../概念/记忆/)
- [聊天模型](https://js.langchain.com/docs/concepts/chat_models/)

LangGraph 允许您跨**不同的[线程](../../concepts/persistence/#threads)** 保存数据。例如，您可以将有关用户的信息（他们的姓名或首选项）存储在共享（跨线程）内存中，并在新线程（例如新对话）中重用它们。

使用[功能API](../../concepts/function_api/)时，您可以使用[Store](/langgraphjs/reference/classes/checkpoint.BaseStore.html)接口将其设置为存储和检索内存：

1. 创建一个 `Store` 的实例
    ```ts
    import { InMemoryStore } from "@langchain/langgraph";
    
    const store = new InMemoryStore();
    ```
2. 将 `store` 实例传递给 `entrypoint()` 包装函数。它将作为 `config.store` 传递到工作流程。
    ```ts
    import { entrypoint } from "@langchain/langgraph";
    
    const workflow = entrypoint({
      store,
      name: "myWorkflow",
    }, async (input, config) => {
      const foo = await myTask({input, store: config.store});
      ...
    });
    ```
在本指南中，我们将展示如何构建和使用具有使用 [Store](/langgraphjs/reference/classes/checkpoint.BaseStore.html) 接口实现的共享内存的工作流程。

!!!提示“注意”

如果您需要向 `StateGraph` 添加跨线程持久性，请查看此[操作指南](../cross-thread-persistence)。

## 设置

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42`。

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/openai @langchain/anthropic @langchain/core uuid
```
接下来，我们需要为 Anthropic 和 OpenAI 设置 API 密钥（我们将使用的 LLM 和嵌入）：
```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
process.env.ANTHROPIC_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

## 示例：具有长期记忆的简单聊天机器人

### 定义商店

在此示例中，我们将创建一个能够检索有关用户首选项的信息的工作流程。我们将通过定义一个 `InMemoryStore` 来实现这一点 - 一个可以在内存中存储数据并查询该数据的对象。

使用 `Store` 接口存储对象时，您定义两件事：

* 对象的命名空间，一个元组（类似于目录）
* 对象键（类似于文件名）

在我们的示例中，我们将使用 `["memories", <user_id>]` 作为命名空间，并使用随机 UUID 作为每个新内存的密钥。

我们首先定义我们的商店：

In [1]:
import { InMemoryStore } from "@langchain/langgraph";
import { OpenAIEmbeddings } from "@langchain/openai";

const inMemoryStore = new InMemoryStore({
  index: {
    embeddings: new OpenAIEmbeddings({
      model: "text-embedding-3-small",
    }),
    dims: 1536,
  },
});

### 创建工作流程

现在让我们创建我们的工作流程：

In [2]:
import { v4 } from "uuid";
import { ChatAnthropic } from "@langchain/anthropic";
import {
  entrypoint,
  task,
  MemorySaver,
  addMessages,
  type BaseStore,
  getStore,
} from "@langchain/langgraph";
import type { BaseMessage, BaseMessageLike } from "@langchain/core/messages";

const model = new ChatAnthropic({
  model: "claude-3-5-sonnet-latest",
});

const callModel = task("callModel", async (
  messages: BaseMessage[],
  memoryStore: BaseStore,
  userId: string
) => {
  const namespace = ["memories", userId];
  const lastMessage = messages.at(-1);
  if (typeof lastMessage?.content !== "string") {
    throw new Error("Received non-string message content.");
  }
  const memories = await memoryStore.search(namespace, {
    query: lastMessage.content,
  });
  const info = memories.map((memory) => memory.value.data).join("\n");
  const systemMessage = `You are a helpful assistant talking to the user. User info: ${info}`;
  
  // 如果用户要求模型记住，则存储新的记忆
  if (lastMessage.content.toLowerCase().includes("remember")) {
    // 硬编码用于演示
    const memory = `Username is Bob`;
    await memoryStore.put(namespace, v4(), { data: memory });
  }
  const response = await model.invoke([
    {
      role: "system",
      content: systemMessage 
    },
    ...messages
  ]);
  return response;
});

// 注意：当通过entrypoint()创建工作流程时，我们在此处传递商店对象
const workflow = entrypoint({
  checkpointer: new MemorySaver(),
  store: inMemoryStore,
  name: "workflow",
}, async (params: {
  messages: BaseMessageLike[];
  userId: string;
}, config) => {
  const messages = addMessages([], params.messages)
  const response = await callModel(messages, config.store, params.userId);
  return entrypoint.final({
    value: response,
    save: addMessages(messages, response),
  });
});

当前存储作为入口点第二个参数的一部分传入，如 `config.store`。

!!!注释 注释

如果您使用 LangGraph Cloud 或 LangGraph Studio，您__不需要__将存储传递到入口点，因为它是自动完成的。

### 运行工作流程！

现在让我们在配置中指定一个用户 ID 并告诉模型我们的名字：

In [3]:
const config = {
  configurable: {
    thread_id: "1",
  },
  streamMode: "values" as const,
};

const inputMessage = {
  role: "user",
  content: "Hi! Remember: my name is Bob",
};

const stream = await workflow.stream({ messages: [inputMessage], userId: "1" }, config);

for await (const chunk of stream) {
  console.log(chunk);
}

AIMessage {
  "id": "msg_01U4xHvf4REPSCGWzpLeh1qJ",
  "content": "Hi Bob! Nice to meet you. I'll remember that your name is Bob. How can I help you today?",
  "additional_kwargs": {
    "id": "msg_01U4xHvf4REPSCGWzpLeh1qJ",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20241022",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 28,
      "cache_creation_input_tokens": 0,
      "cache_read_input_tokens": 0,
      "output_tokens": 27
    }
  },
  "response_metadata": {
    "id": "msg_01U4xHvf4REPSCGWzpLeh1qJ",
    "model": "claude-3-5-sonnet-20241022",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 28,
      "cache_creation_input_tokens": 0,
      "cache_read_input_tokens": 0,
      "output_tokens": 27
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 28,
    "outp

In [4]:
const config2 = {
  configurable: {
    thread_id: "2",
  },
  streamMode: "values" as const,
};

const followupStream = await workflow.stream({
  messages: [{
    role: "user",
    content: "what is my name?",
  }],
  userId: "1"
}, config2);

for await (const chunk of followupStream) {
  console.log(chunk);
}

AIMessage {
  "id": "msg_01LB4YapkFawBUbpiu3oeWbF",
  "content": "Your name is Bob.",
  "additional_kwargs": {
    "id": "msg_01LB4YapkFawBUbpiu3oeWbF",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20241022",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 28,
      "cache_creation_input_tokens": 0,
      "cache_read_input_tokens": 0,
      "output_tokens": 8
    }
  },
  "response_metadata": {
    "id": "msg_01LB4YapkFawBUbpiu3oeWbF",
    "model": "claude-3-5-sonnet-20241022",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 28,
      "cache_creation_input_tokens": 0,
      "cache_read_input_tokens": 0,
      "output_tokens": 8
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 28,
    "output_tokens": 8,
    "total_tokens": 36,
    "input_token_details": {
     

我们现在可以检查内存存储并验证我们实际上已经为用户保存了内存：

In [5]:
const memories = await inMemoryStore.search(["memories", "1"]);
for (const memory of memories) {
  console.log(memory.value);
}

{ data: 'Username is Bob' }


现在让我们为另一个用户运行工作流程，以验证有关第一个用户的记忆是否是独立的：

In [6]:
const config3 = {
  configurable: {
    thread_id: "3",
  },
  streamMode: "values" as const,
};

const otherUserStream = await workflow.stream({
  messages: [{
    role: "user",
    content: "what is my name?",
  }],
  userId: "2"
}, config3);

for await (const chunk of otherUserStream) {
  console.log(chunk);
}

AIMessage {
  "id": "msg_01KK7CweVY4ZdHxU5bPa4skv",
  "content": "I don't have any information about your name. While I aim to be helpful, I can only know what you directly tell me during our conversation.",
  "additional_kwargs": {
    "id": "msg_01KK7CweVY4ZdHxU5bPa4skv",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20241022",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 25,
      "cache_creation_input_tokens": 0,
      "cache_read_input_tokens": 0,
      "output_tokens": 33
    }
  },
  "response_metadata": {
    "id": "msg_01KK7CweVY4ZdHxU5bPa4skv",
    "model": "claude-3-5-sonnet-20241022",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 25,
      "cache_creation_input_tokens": 0,
      "cache_read_input_tokens": 0,
      "output_tokens": 33
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "u